In [ ]:
#gemma

#  causality sentence Identifier

## 1.  load sentence

In [15]:
import ast
import pandas as pd

#load semeval data with embed
pm_DB = pd.read_csv('data/in/KNN_RAG/SCITE_TrueFalse.csv')
instance_emb = pm_DB['embedings'].to_list()
instance_emb = [ast.literal_eval(item) for item in instance_emb]

instances = pm_DB['sentence'].to_list()
len(instances)

786

## 2. load fewshot database

In [16]:
#load fewshotDB with embed
semeval_data = pd.read_csv('data/in/KNN_RAG/comb_SCITEsemADE_CausalityPattern.csv')
fewshot_emb = semeval_data['embedings'].to_list()
fewshot_emb = [ast.literal_eval(item) for item in fewshot_emb]

examples = semeval_data['sentence'].to_list()


## 3. get causality label for sentence

In [17]:
import groq
from groq import Groq

client = groq.Groq(api_key= "enter your API key here..")
#gemma2-9b-it# llama3-8b-8192


In [18]:
#gemma2-9b-it# llama3-8b-8192  mixtral-8x7b-32768
def CS_Identifier(causality_sentence, fewshot_examples):

    chat_completion = client.chat.completions.create(
    #
    # Required parameters
    #
    messages=[
        # Set an optional system message. This sets the behavior of the
        # assistant and can be used to provide specific instructions for
        # how it should behave throughout the conversation.
        {
            "role": "system",
            "content": """
            You will be given a sentence, and your task is to identify whether the sentence shows a 
          causality relation or not. If the sentence shows a causality relation, you can output 1; if it doesn't, output 0. 
          Your output should be in json format: {output :  }.
          
          Hint: If the sentence shows causality relation, you will be able to identify a pair of 
          phrase in the sentence where one phrase will be the cause of the other phrase(effect)
          
         Below are 10 example sentences which shows causality relation  with  their cause and effect 
          phrase delimited between the XML tags <cause>, </cause> , <effect> and </effect>.
           """+ fewshot_examples
        },
        {
          "role": "user",
          "content": input_sentence 
        },
    ],

    # The language model which will generate the completion.
    model="mixtral-8x7b-32768",
    temperature=0.0,
    seed = 1,
    max_tokens=100,
    top_p=1,
    response_format={"type": "json_object"},)
    out = chat_completion.choices[0].message.content
    
    return out

In [19]:
import json
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

causality_identifier=[]
fewshotfound_inputsentence_index=[]
for i in range (len(pm_DB)):
    input_sentence = pm_DB['sentence'][i]
    # few shot examples augmentation.
   
    ##########KNN###########
    # Combine lists into a numpy array
    lists = np.array(fewshot_emb)

    # Calculate cosine similarity between list_1 and other lists
    similarities = cosine_similarity([instance_emb[i]], fewshot_emb).flatten()

    # Get indices of the two most similar lists
    most_similar_indices = similarities.argsort()[-10:][::-1]

    # Print the most similar lists and their cosine similarity values

    fewshot_examples = [examples[j] for j in most_similar_indices]

    fewshot_examples = {f'example {k + 1}': sentence for k, sentence in enumerate(fewshot_examples)}
    fewshot_examples = json.dumps(fewshot_examples)   

    #####################
    
    #print('input_sentence : ', input_sentence)

    #print('fewshot_index : ', fewshot_index)
    
    try:
        # output generation
        out = CS_Identifier(input_sentence, fewshot_examples)
        print('out', out)

    
        json_out = json.loads(out)
        causality_identifier_out = json_out['output']
        causality_identifier.append(int(causality_identifier_out))
        
    except TypeError as e:
        causality_identifier.append(0)
        continue
        
    except KeyError as e:
        causality_identifier.append(0)
        continue
    
    except groq.APIConnectionError as e:
        causality_identifier.append(0)
        continue
    except groq.RateLimitError as e:
        print('####################################ratelimit')
        causality_identifier.append(0)
        continue
    except groq.APIStatusError as e:
        causality_identifier.append(0)
        continue
    
    
    print(i)
    print(causality_identifier_out)
    
    

out {"output": 1, "cause": "this statement", "effect": "people into chaos"}
0
1
out {"output": 1, "cause": "the stress and hard work", "effect": "a bleeding stomach ulcer"}
1
1
out {"output": 0}
2
0
out {"output": 1, "cause": "hormone deficiencies and imbalances", "effect": "Paralysis or convulsions"}
3
1
out {"output": 1}
4
1
out {"output": 1, "cause": "sunlight and glare", "effect": "eye strain and fatigue"}
5
1
out {"output": 0}
6
0
out {"output": "0"}
7
0
out {"output": 0}
8
0
out {"output": 0}
9
0
out {"output": 0}
10
0
out {"output": 1, "cause": "rapid power acceleration", "effect": "pressure"}
11
1
out {"output": 0}
12
0
out {"output": 0}
13
0
out {
"output": 0
}
14
0
out {"output": 0}
15
0
out {"output": 0}
16
0
out {"output": 1, "cause": "higher radiation doses", "effect": "implants inserted into bone by several authors"}
17
1
out {
"output": 1
}
18
1
out {"output": 0}
19
0
out {"output": 0}
20
0
out {"output": 1, "cause": "intoxication from alcohol or illicit drug use", "effe

out {
"example 1": 1,
"example 2": 1,
"example 3": 1,
"example 4": 1,
"example 5": 1,
"example 6": 1,
"example 7": 1,
"example 8": 1,
"example 9": 1,
"example 10": 1
}
out {"output": 1, "cause": "a songwriter", "effect": "writes the lyrics, as well as the musical composition or melody to songs"}
199
1
out {"output": 1}
200
1
out {"output": 1, "cause_phrase": "the latter course", "effect_phrase": "explained the biological phenomena in a mathematical way"}
201
1
out {"output": 0}
202
0
out {"output": 0}
203
0
out {"output": 0}
204
0
out {"output": 0}
205
0
out {"output": 0}
206
0
out {"output": 0}
207
0
out {"output": 1, "cause": "on a timelock and would not have opened until 9am", "effect": "the money was locked in a safe"}
208
1
out {"output": 0}
209
0
out {"output": 1}
211
1
out {"output": 0}
212
0
out {"output": 0}
213
0
out {"output": 1}
214
1
out {"output": 0}
215
0
out {"output": 0}
216
0
out {"output": 0}
217
0
out {"output": 1, "cause": "invested successfully", "effect": "the re

out {"output": "0"}
383
0
out {"output": 0}
384
0
out {"output": 0}
385
0
out {"output": 0}
386
0
out {"output": 0}
387
0
out {"output": 1}
388
1
out {
"example 1": 1,
"example 2": 1,
"example 3": 1,
"example 4": 1,
"example 5": 1,
"example 6": 1,
"example 7": 1,
"example 8": 1,
"example 9": 1,
"example 10": 1
}
out {"output": 1, "cause": "thin transparent layers are juxtaposed with thicker juicier layers of paint", "effect": "Each painting is built up of many layers over time"}
390
1
out {"output": 0}
391
0
out {"output": "0"}
392
0
out {"output": 0}
393
0
out {"output": 0}
394
0
out {"output": 1}
395
1
out {"output": 1}
396
1
out {"output": 1}
397
1
out {"output": 1}
398
1
out {"output": "0"}
399
0
out {"output": 0}
400
0
out {"output": 1}
401
1
out {"output": 0}
402
0
out {"output": 0}
403
0
out {"output": 1}
404
1
out {"output": 1}
405
1
out {"output": 0}
406
0
out {"output": 1}
407
1
out {"output": 0}
408
0
out {"output": 0}
409
0
out {"output": 0}
410
0
out {"output": 0}
411
0
ou

out {"output": 0}
598
0
out {"output": 1, "cause": "the appointment of Donald Sumner, formerly Conservative MP for Orpington, to be a county court judge", "effect": "The election"}
599
1
out {"output": 0}
600
0
out {"output": 0}
601
0
out {"output": 1, "cause": "ethical issues involved", "effect": "controversy and even heated debate"}
602
1
out {"output": 1}
603
1
out {"output": 0}
604
0
out {"output": 0}
605
0
out {"output": 0}
606
0
out {"output": 1}
607
1
out {"output": 1, "cause": "The hull caused a scratch on the eye", "effect": "that is why it's not healing all the way"}
608
1
out {"output": 1}
609
1
out {"output": 0}
610
0
out {
"output": "0"
}
611
0
out {
"output": 1
}
612
1
out {"output": 1, "cause": "a politician", "effect": "uses tourism as an instrument for power"}
613
1
out {"output": 0}
614
0
out {
"output": 1,
"cause": "fuel leak in one of the C-17s",
"effect": "a delay on the morning of Nov. 7 as the crew was preparing for departure for Cairo, Egypt"
}
615
1
out {"outpu

out {
"output": 1,
"cause": "an electrical short at the Farragut North station",
"effect": "a fire on the tracks which shut down service three times over the course of the day"
}
776
1
out {"output": 1}
777
1
out {"output": 1}
778
1
out {"output": 1}
779
1
out {
"output": 1
}
780
1
out {"output": 1, "cause_phrase": "the company told employees an elaborate story about its creation", "effect_phrase": "Before introducing Via in three test markets"}
781
1
out {"output": 0}
782
0
out {"output": 0}
783
0
out {"output": 1}
784
1
out {"output": 0}
785
0


In [20]:
len(causality_identifier)

786

In [21]:
len(causality_identifier)

786

In [22]:
len(fewshotfound_inputsentence_index)

0

In [23]:
#get  pattern_matched label
pattern_matched = [0]*(len(pm_DB))

# Update values to 1 at specified indexes
for index in fewshotfound_inputsentence_index:
    if index < len(pattern_matched):
        pattern_matched[index] = 1
pattern_matched

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [24]:
pm_DB['GPT_causality_label'] = causality_identifier
pm_DB['patternmatcher_label'] = pattern_matched
pm_DB

,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,patternmatcher_label
0,0,0,This statement threw people into chaos .,0,"[-0.0067518032155931, -0.003759827697649598, -...",1,0
1,1,1,Cutler ended up with a bleeding stomach ulcer...,1,"[-0.003396878018975258, -0.009281124919652939,...",1,0
2,2,2,This theme has been covered in science fict...,0,"[0.03351648524403572, -0.02911864034831524, 0....",0,0
3,3,3,Paralysis or convulsions are caused by <e3...,1,"[-0.023387493565678596, -0.02253008261322975, ...",1,0
4,4,4,"In economic terms, the ecological catastrophe...",1,"[0.008494839072227478, -0.038574811071157455, ...",1,0
...,...,...,...,...,...,...,...
781,781,781,"Before introducing Via in three test markets, ...",0,"[0.0033979706931859255, -0.009512247517704964,...",1,0
782,782,782,A programmer uses a high level language to...,0,"[0.011056014336645603, 0.002396157244220376, -...",0,0
783,783,783,One of the most vivid portraits is that of Bro...,0,"[-0.02381613664329052, -0.018164167180657387, ...",0,0
784,784,784,The earthquake caused the failures of the ...,1,"[-0.025205399841070175, 0.004254561383277178, ...",1,0


In [25]:
# Function to set 'prediction' column based on conditions
def set_prediction(row):
    if row['GPT_causality_label'] == 1 and row['patternmatcher_label'] == 1:
        return 1
    else:
        return 0

# Apply the function to each row and create the 'prediction' column
pm_DB['prediction'] = pm_DB.apply(set_prediction, axis=1)
pm_DB

,Unnamed: 0.1,Unnamed: 0,sentence,label,embedings,GPT_causality_label,patternmatcher_label,prediction
0,0,0,This statement threw people into chaos .,0,"[-0.0067518032155931, -0.003759827697649598, -...",1,0,0
1,1,1,Cutler ended up with a bleeding stomach ulcer...,1,"[-0.003396878018975258, -0.009281124919652939,...",1,0,0
2,2,2,This theme has been covered in science fict...,0,"[0.03351648524403572, -0.02911864034831524, 0....",0,0,0
3,3,3,Paralysis or convulsions are caused by <e3...,1,"[-0.023387493565678596, -0.02253008261322975, ...",1,0,0
4,4,4,"In economic terms, the ecological catastrophe...",1,"[0.008494839072227478, -0.038574811071157455, ...",1,0,0
...,...,...,...,...,...,...,...,...
781,781,781,"Before introducing Via in three test markets, ...",0,"[0.0033979706931859255, -0.009512247517704964,...",1,0,0
782,782,782,A programmer uses a high level language to...,0,"[0.011056014336645603, 0.002396157244220376, -...",0,0,0
783,783,783,One of the most vivid portraits is that of Bro...,0,"[-0.02381613664329052, -0.018164167180657387, ...",0,0,0
784,784,784,The earthquake caused the failures of the ...,1,"[-0.025205399841070175, 0.004254561383277178, ...",1,0,0


In [26]:
#get accuracy for fewshot_semeval_causalityDetection: b/w label and GPT_causality_label 

from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

# Calculate precision, recall, and F1 score
precision = precision_score(pm_DB['label'], pm_DB['GPT_causality_label'])
recall = recall_score(pm_DB['label'], pm_DB['GPT_causality_label'])
f1 = f1_score(pm_DB['label'], pm_DB['GPT_causality_label'])
accuracy = accuracy_score(pm_DB['label'], pm_DB['GPT_causality_label'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')


Accuracy: 0.8409669211195929
F1 Score: 0.745417515274949
Precision: 0.61
Recall: 0.9581151832460733


In [27]:

#filtered_rows = pm_DB[(pm_DB['GPT_causality_label'] == 1) & (pm_DB['patternmatcher_label'] == 1)]
#filtered_rows.to_csv('data/out/alzeimers_causalityrelation.csv')

#pm_DB.to_csv('data/out/RAG_causality/semeval/fewshotcomb_semeval_causalityDetection.csv')

In [28]:
pm_DB.to_csv('data/out/RAG_causality/task1/gemma/KNN_ADE_causalityDetection.csv')#RAG, rand,zero, ADE, SCITE, sem